In [22]:
import pandas as pd

df=pd.read_csv("./data/9988.HK_5m_test_fmt.csv")



In [ ]:
df['timestamps'] = pd.to_datetime(df['timestamps']).dt.strftime('%Y/%-m/%-d %-H:%M')


In [27]:
df

,timestamps,open,high,low,close,volume,amount
0,2025/12/1 9:30,152.899994,154.199997,152.000000,154.000000,6899471,1.062519e+09
1,2025/12/1 9:35,154.000000,154.100006,153.300003,153.899994,2915569,4.487061e+08
2,2025/12/1 9:40,154.000000,155.600006,153.899994,155.199997,5721342,8.879523e+08
3,2025/12/1 9:45,155.300003,156.100006,154.899994,156.000000,5192700,8.100612e+08
4,2025/12/1 9:50,156.000000,156.399994,155.800003,156.199997,3633324,5.675252e+08
...,...,...,...,...,...,...,...
665,2025/12/12 15:40,153.000000,153.199997,152.899994,153.100006,1792100,2.743705e+08
666,2025/12/12 15:45,153.000000,153.100006,152.800003,152.800003,1793500,2.740468e+08
667,2025/12/12 15:50,152.800003,153.199997,152.800003,152.800003,1546445,2.362968e+08
668,2025/12/12 15:55,152.800003,153.000000,152.399994,153.000000,2620600,4.009518e+08


In [ ]:
# 设置df timestamps字符串格式


In [ ]:
df.set_index('timestamps', inplace=True)


In [ ]:
df.to_csv("./data/9988.HK_5m_fmt.csv")
